In [10]:
import torchio as tio
import numpy as np
import os
import cv2

# Define refined augmentation pipeline without RandomElasticDeformation
transform = tio.Compose([
    tio.RandomBlur(p=0.2),  # Probability of 20%
    tio.RandomNoise(p=0.2),  # Probability of 20%
    tio.RandomAffine(scales=(0.98, 1.02), degrees=2, isotropic=True, translation=(0, 0, 0), p=0.2),  # Limited affine transformation
    tio.RandomBiasField(coefficients=0.1, p=0.2),  # Reduced coefficients
    tio.RandomMotion(degrees=2, translation=2, num_transforms=1, p=0.2)  # Reduced degrees and translation
])

# Function to augment and save images
def augment_images(image_dir, save_dir, target_count=2000):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    image_names = os.listdir(image_dir)
    num_images = len(image_names) 
    num_augmentations = target_count // num_images
    additional_images = target_count % num_images

    for image_name in image_names:
        image_path = os.path.join(image_dir, image_name)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = np.expand_dims(image, axis=(0, 1))  # Add batch and channel dimensions
        subject = tio.Subject(
            mri=tio.ScalarImage(tensor=image)
        )
        
        for i in range(num_augmentations):
            augmented_subject = transform(subject)
            augmented_image = augmented_subject['mri'].numpy().squeeze()
            augmented_image_name = f"{os.path.splitext(image_name)[0]}_aug_{i}.png"
            augmented_image_path = os.path.join(save_dir, augmented_image_name)
            cv2.imwrite(augmented_image_path, augmented_image)

    # Augment additional images to meet the target count
    for i in range(additional_images):
        image_name = image_names[i % num_images]
        image_path = os.path.join(image_dir, image_name)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = np.expand_dims(image, axis=(0, 1))  # Add batch and channel dimensions
        subject = tio.Subject(
            mri=tio.ScalarImage(tensor=image)
        )
        
        augmented_subject = transform(subject)
        augmented_image = augmented_subject['mri'].numpy().squeeze()
        augmented_image_name = f"{os.path.splitext(image_name)[0]}_extra_aug_{i}.png"
        augmented_image_path = os.path.join(save_dir, augmented_image_name)
        cv2.imwrite(augmented_image_path, augmented_image)

# Example usage
no_tumor_image_dir = 'combined/train/no_tumor'
augmented_no_tumor_image_dir = 'combined/train/aug_no_tumor'
augment_images(no_tumor_image_dir, augmented_no_tumor_image_dir, target_count=2000)
